## Calculate metrics

Run the segment below first, then the following can run normally. What the hell is this?

In [5]:
import numpy as np

gt = np.load(r'./data_fi3d/emb/emb_gt.npy')
gen = np.load(r'./data_fi3d/emb/emb_gen_none_None.npy')
genp = np.load(r'./data_fi3d/emb/emb_gen_gaussian_1.npy')

gt = np.mean(gt, axis=1)
gen = np.mean(gen, axis=1)
genp = np.mean(genp, axis=1)

gt = np.delete(gt, [487, 1894], axis=0)
gen = np.delete(gen, [487, 1894], axis=0)
genp = np.delete(genp, [487, 1894], axis=0)

from sklearn.decomposition import PCA, IncrementalPCA


def apply_pca(data, n_components=10):
    """
    Apply PCA to reduce the dimensionality of the data.

    Args:
        data: numpy.ndarray, dataset to be transformed.
        n_components: int, number of principal components to retain.

    Returns:
        numpy.ndarray: Transformed dataset with reduced dimensions.
    """
    
    pca = IncrementalPCA(n_components=n_components, batch_size=16)
    # pca = PCA(n_components=n_components)
    data_reduced = pca.fit_transform(data)
    
    return data_reduced

apply_pca(gen)

array([[-1.37157156,  2.14924841, -3.05180756, ...,  0.19422355,
        -0.05954894, -0.12712566],
       [-1.6478645 ,  0.97119381, -1.35586851, ...,  0.21376756,
         0.05664569, -0.19794771],
       [-0.0712567 ,  1.02115189,  0.4101954 , ..., -0.16195227,
         0.01179378, -0.08877666],
       ...,
       [ 2.40670481, -1.86114916,  3.0128795 , ...,  0.06596056,
        -0.08211737,  0.04486519],
       [ 0.03742224, -7.77419999,  2.77032292, ..., -0.01122986,
        -0.11048092, -0.17316445],
       [ 1.97254642, -4.70035634,  1.17356833, ..., -0.04575332,
        -0.22742945, -0.17202515]])

In [4]:
import os
import numpy as np
import pandas as pd
from os.path import join as pjoin
from tqdm import tqdm

import torch
import numpy as np
from sklearn.mixture import GaussianMixture
import ot   # pip install POT if missing

from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA

class Evaluator:
    def __init__(self, gt, gen, genp):
        gt, rows = self.del_nan(np.mean(gt, axis=1))
        gen, _ = self.del_nan(np.mean(gen, axis=1), rows)
        genp, _ = self.del_nan(np.mean(genp, axis=1), rows)
        
        # print(gt.shape, gen.shape, genp.shape)  # (4608, 512) (4608, 512)
        
        self.gt = gt
        self.gen = gen
        self.genp = genp

    def eval(self):
        print("FID ----------------------->")
        fid = self.fid(self.gt, self.gen)
        print("FIDp ----------------------->")
        fidp = self.fid(self.gt, self.genp)
        print("MW2 ----------------------->")
        mw2 = self.MW2(self.gt, self.gen)
        print("MW2p ----------------------->")
        mw2p = self.MW2(self.gt, self.genp)
        
        print(f'FID: {fid} -> {fidp}')
        print(f'MW2: {mw2} -> {mw2p}')
        
        return fid, fidp, mw2, mw2p
    
    def fid(self, real, fake):
        """
        Frechet Inception Distance
        """
        n = real.shape[0]

        mean_real = np.mean(real, axis=0)[None, ...]
        mean_fake = np.mean(fake, axis=0)[None, ...]

        cov_fake_sum = fake.T @ fake - n * mean_fake.T @ mean_fake
        cov_real_sum = real.T @ real - n * mean_real.T @ mean_real

        cov_fake = cov_fake_sum / (n - 1)
        cov_real = cov_real_sum / (n - 1)
                
        mu1, mu2, sig1, sig2 = mean_fake, mean_real, cov_fake, cov_real

        mu1_tensor = torch.tensor(mu1)
        mu2_tensor = torch.tensor(mu2)
        sig1_tensor = torch.tensor(sig1)
        sig2_tensor = torch.tensor(sig2) 

        a = (mu1_tensor - mu2_tensor).square().sum(dim=-1)
        b = sig1_tensor.trace() + sig2_tensor.trace()
        c = torch.linalg.eigvals(sig1_tensor @ sig2_tensor).sqrt().real.sum(dim=-1)

        score = a + b - 2 * c

        return score.item()
    
    def MW2(self, dist1, dist2, K=13):
        """
        Compute the Wasserstein distance between two distributions, each represented as
        a multivariate Gaussian Mixture Model (GMM).

        Args:
            dist1: the first distribution, shape (N, D)
            dist2: the second distribution, shape (N, D)
            K: int, number of components in the GMM
        
        Returns:
            float: The Wasserstein-2 distance between the two GMMs.
        """
        
        # Apply PCA to reduce the dimensionality of the data
        print("         PCA ----------------------->")
        dist1_pca = self.apply_pca(dist1)
        dist2_pca = self.apply_pca(dist2)
        
        print("         GMM ----------------------->")
        # Fit GMMs to the input distributions
        gmm1 = GaussianMixture(n_components=K, random_state=0, max_iter=200).fit(dist1_pca)
        gmm2 = GaussianMixture(n_components=K, random_state=0, max_iter=200).fit(dist2_pca)
        
        # Compute the pairwise Euclidean distances between the means of the GMM components
        C = ot.dist(gmm1.means_, gmm2.means_, metric='euclidean')

        # print(f'gmm1 weights: {gmm1.weights_}, gmm2 weights: {gmm2.weights_}, gmm1 means: {gmm1.means_}, gmm2 means: {gmm2.means_}')
        
        # Normalize the cost matrix to prevent numerical instability
        C /= C.max()

        # Compute the optimal transport plan using the Earth Mover's Distance (EMD) algorithm
        gamma = ot.emd(gmm1.weights_, gmm2.weights_, C)

        # Calculate the Wasserstein distance using the transport plan and the cost matrix
        W2 = np.sum(gamma * C)

        return W2
    
    def apply_pca(self, data, n_components=10):
        """
        Apply PCA to reduce the dimensionality of the data.

        Args:
            data: numpy.ndarray, dataset to be transformed.
            n_components: int, number of principal components to retain.

        Returns:
            numpy.ndarray: Transformed dataset with reduced dimensions.
        """
        
        # pca = PCA(n_components=n_components)
        pca = IncrementalPCA(n_components=n_components, batch_size=16)
        
        data_reduced = pca.fit_transform(data)
        
        return data_reduced
        
    def del_nan(self, array, rm_rows=None):
        if rm_rows == None:
            rm_rows = []
            if np.isnan(array).any():
                for i in range(array.shape[0]):
                    for j in range(array.shape[1]):
                        if np.isnan(array[i][j]):
                            rm_rows.append(i)
                            break
            array = np.delete(array, rm_rows, axis=0)
        else:
            array = np.delete(array, rm_rows, axis=0)
                        
        return array, rm_rows

if __name__=="__main__":
    emb_dir = './data_fi3d/emb'
    flist = os.listdir(emb_dir)
    perturbs = [f.strip('.npy').split('_')[-2] + '_' + f.strip('.npy').split('_')[-1] for f in flist if 'emb_gen' in f]
    
    gt = np.load('./data_fi3d/emb/emb_gt.npy')
    gen = np.load('./data_fi3d/emb/emb_gen_none_None.npy')
    
    res_dict = {'method': [], 'FID': [], 'MW2': [], 'FIDp': [], 'MW2p': []}
    for perturb in perturbs:
        print(f'Evaluating {perturb} motions --------------> ')
        genp = np.load(f'./data_fi3d/emb/emb_gen_{perturb}.npy')
        
        evaluator = Evaluator(gt, gen, genp)
        fid, fidp, mw2, mw2p = evaluator.eval()
        
        res_dict['method'].append(perturb)
        res_dict['FID'].append(fid)
        res_dict['MW2'].append(mw2)
        res_dict['FIDp'].append(fidp)
        res_dict['MW2p'].append(mw2p)

    df = pd.DataFrame(res_dict)
    df.to_csv('eval_results.csv', index=False)

Evaluating repeating_quater motions --------------> 
FID ----------------------->
FIDp ----------------------->
MW2 ----------------------->
         PCA ----------------------->
         GMM ----------------------->
MW2p ----------------------->
         PCA ----------------------->
         GMM ----------------------->
FID: 0.5587005615234375 -> 0.4398956298828125
MW2: 0.12026208655477956 -> 0.12934075743994108


## computational efficiency test

In [8]:
import os
import numpy as np
import pandas as pd
from os.path import join as pjoin
from tqdm import tqdm

import time

import torch
import numpy as np
from sklearn.mixture import GaussianMixture
import ot   # pip install POT if missing

from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA

class Evaluator:
    def __init__(self, gt, gen, genp):
        gt, rows = self.del_nan(np.mean(gt, axis=1))
        gen, _ = self.del_nan(np.mean(gen, axis=1), rows)
        genp, _ = self.del_nan(np.mean(genp, axis=1), rows)
        
        # print(gt.shape, gen.shape, genp.shape)  # (4608, 512) (4608, 512)
        
        self.gt = gt
        self.gen = gen
        self.genp = genp

    def eval(self):
        start_time = time.time()
        # print("FID ----------------------->")
        fid = self.fid(self.gt, self.gen)
        # print("FIDp ----------------------->")
        fidp = self.fid(self.gt, self.genp)
        efid = time.time() - start_time
        
        start_time = time.time()
        gt_pca = self.apply_pca(self.gt)    # PCA 最好放在MW2里面  这里放在外面是为了计算时间
        gen_pca = self.apply_pca(self.gen)
        gt_pca = self.apply_pca(self.gt)
        genp_pca = self.apply_pca(self.genp)
        epca = time.time() - start_time
        
        start_time = time.time()
        # print("MW2 ----------------------->")
        mw2 = self.MW2(gt_pca, gen_pca)
        # print("MW2p ----------------------->")
        mw2p = self.MW2(gt_pca, genp_pca)
        emw2 = time.time() - start_time
        
        print(f'FID: {fid} -> {fidp}')
        print(f'MW2: {mw2} -> {mw2p}')
        
        return fid, fidp, mw2, mw2p, efid, epca, emw2
    
    def fid(self, real, fake):
        """
        Frechet Inception Distance
        """
        n = real.shape[0]

        mean_real = np.mean(real, axis=0)[None, ...]
        mean_fake = np.mean(fake, axis=0)[None, ...]

        cov_fake_sum = fake.T @ fake - n * mean_fake.T @ mean_fake
        cov_real_sum = real.T @ real - n * mean_real.T @ mean_real

        cov_fake = cov_fake_sum / (n - 1)
        cov_real = cov_real_sum / (n - 1)
                
        mu1, mu2, sig1, sig2 = mean_fake, mean_real, cov_fake, cov_real

        mu1_tensor = torch.tensor(mu1)
        mu2_tensor = torch.tensor(mu2)
        sig1_tensor = torch.tensor(sig1)
        sig2_tensor = torch.tensor(sig2) 

        a = (mu1_tensor - mu2_tensor).square().sum(dim=-1)
        b = sig1_tensor.trace() + sig2_tensor.trace()
        c = torch.linalg.eigvals(sig1_tensor @ sig2_tensor).sqrt().real.sum(dim=-1)

        score = a + b - 2 * c

        return score.item()
    
    def MW2(self, dist1_pca, dist2_pca, K=13):
        """
        Compute the Wasserstein distance between two distributions, each represented as
        a multivariate Gaussian Mixture Model (GMM).

        Args:
            dist1: the first distribution, shape (N, D)
            dist2: the second distribution, shape (N, D)
            K: int, number of components in the GMM
        
        Returns:
            float: The Wasserstein-2 distance between the two GMMs.
        """
        
        # Apply PCA to reduce the dimensionality of the data
        # print("         PCA ----------------------->")
        # dist1_pca = self.apply_pca(dist1)
        # dist2_pca = self.apply_pca(dist2)
        
        # print("         GMM ----------------------->")
        # Fit GMMs to the input distributions
        gmm1 = GaussianMixture(n_components=K, random_state=0, max_iter=200).fit(dist1_pca)
        gmm2 = GaussianMixture(n_components=K, random_state=0, max_iter=200).fit(dist2_pca)
        
        # Compute the pairwise Euclidean distances between the means of the GMM components
        C = ot.dist(gmm1.means_, gmm2.means_, metric='euclidean')

        # print(f'gmm1 weights: {gmm1.weights_}, gmm2 weights: {gmm2.weights_}, gmm1 means: {gmm1.means_}, gmm2 means: {gmm2.means_}')
        
        # Normalize the cost matrix to prevent numerical instability
        C /= C.max()

        # Compute the optimal transport plan using the Earth Mover's Distance (EMD) algorithm
        gamma = ot.emd(gmm1.weights_, gmm2.weights_, C)

        # Calculate the Wasserstein distance using the transport plan and the cost matrix
        W2 = np.sum(gamma * C)

        return W2
    
    def apply_pca(self, data, n_components=10):
        """
        Apply PCA to reduce the dimensionality of the data.

        Args:
            data: numpy.ndarray, dataset to be transformed.
            n_components: int, number of principal components to retain.

        Returns:
            numpy.ndarray: Transformed dataset with reduced dimensions.
        """
        
        # pca = PCA(n_components=n_components)
        pca = IncrementalPCA(n_components=n_components, batch_size=16)
        
        data_reduced = pca.fit_transform(data)
        
        return data_reduced
        
    def del_nan(self, array, rm_rows=None):
        if rm_rows == None:
            rm_rows = []
            if np.isnan(array).any():
                for i in range(array.shape[0]):
                    for j in range(array.shape[1]):
                        if np.isnan(array[i][j]):
                            rm_rows.append(i)
                            break
            array = np.delete(array, rm_rows, axis=0)
        else:
            array = np.delete(array, rm_rows, axis=0)
                        
        return array, rm_rows

if __name__=="__main__":
    emb_dir = './data_fi3d/emb'
    flist = os.listdir(emb_dir)
    perturbs = [f.strip('.npy').split('_')[-2] + '_' + f.strip('.npy').split('_')[-1] for f in flist if 'emb_gen' in f]
    
    gt = np.load('./data_fi3d/emb/emb_gt.npy')
    gen = np.load('./data_fi3d/emb/emb_gen_none_None.npy')
    
    res_dict = {'method': [], 'FID': [], 'MW2': [], 'FIDp': [], 'MW2p': [], 'time_fid': [], 'time_pca':[], 'time_mw2': []}
    for perturb in perturbs:
        print(f'Evaluating {perturb} motions --------------> ')
        genp = np.load(f'./data_fi3d/emb/emb_gen_{perturb}.npy')
        
        evaluator = Evaluator(gt, gen, genp)
        fid, fidp, mw2, mw2p, time_fid, time_pca, time_mw2 = evaluator.eval()
        
        res_dict['method'].append(perturb)
        res_dict['FID'].append(fid)
        res_dict['MW2'].append(mw2)
        res_dict['FIDp'].append(fidp)
        res_dict['MW2p'].append(mw2p)
        res_dict['time_fid'].append(time_fid)
        res_dict['time_pca'].append(time_pca)
        res_dict['time_mw2'].append(time_mw2)

    df = pd.DataFrame(res_dict)
    df.to_csv('eval_results_time.csv', index=False)

Evaluating smoothing_(5, 1) motions --------------> 
FID: 0.5587005615234375 -> 2.466796875
MW2: 0.12026208655477956 -> 0.29983442000055904
Evaluating uniform_1 motions --------------> 
FID: 0.5587005615234375 -> 6.2283172607421875
MW2: 0.12026208655477956 -> 0.30102321897457995
Evaluating shuffling_20 motions --------------> 
FID: 0.5587005615234375 -> 0.7993545532226562
MW2: 0.12026208655477956 -> 0.1507211355209278
Evaluating skipping_5 motions --------------> 
FID: 0.5587005615234375 -> 0.5721282958984375
MW2: 0.12026208655477956 -> 0.12339909020452275
Evaluating gaussian_1 motions --------------> 
FID: 0.5587005615234375 -> 1.79193115234375
MW2: 0.12026208655477956 -> 0.290033033953415
Evaluating smoothing_(1, 1) motions --------------> 
FID: 0.5587005615234375 -> 0.6676406860351562
MW2: 0.12026208655477956 -> 0.12299645395377269
Evaluating gaussian_0.01 motions --------------> 
FID: 0.5587005615234375 -> 0.53924560546875
MW2: 0.12026208655477956 -> 0.12014404223775903
Evaluating 